In [5]:
from config import Config
from model import Transformer
from tokenizer import Tokenizer
import torch as t
import os
import warnings

warnings.filterwarnings("ignore")

In [6]:
def get_latest_model_folder(model_dir, file_start):
    model_files = [file for file in os.listdir(model_dir) if file_start in file]
    model_files.sort()
    return os.path.join(model_dir, model_files[-1])


In [7]:
model_dir = 'saved_runs'
file_start = 'mod_digit_add_'
config = Config()
latest_model_folder = get_latest_model_folder(model_dir, file_start)
latest_model_folder

'saved_runs/mod_digit_add_2024-09-30_18-37-33'

In [8]:
# total models will be num_models + 1
num_models = 10
num_epochs = 2000

first_model = Transformer(config)
first_model.load_state_dict(t.load(latest_model_folder + '/init.pth')['model'])
first_model.eval()


models = [Transformer(config) for _ in range(num_models-1)]
intervals = t.linspace(0, num_epochs, num_models + 1).int()[1:-1]
print(intervals)
weights = [t.load(latest_model_folder + f'/{num}.pth') for num in intervals] 
for i in range(len(weights)):
    models[i].load_state_dict(weights[i]['model'])
    models[i].eval()

last_model = Transformer(config)
last_model.load_state_dict(t.load(latest_model_folder + f'/final.pth')['model'])
last_model.eval()

tokenizer = Tokenizer(config)

tensor([ 200,  400,  600,  800, 1000, 1200, 1400, 1600, 1800],
       dtype=torch.int32)


In [9]:
def get_accuracy(model, maxnum = 100, p = 113):

    correct = 0
    count = 0
    for i in range(maxnum):
        for j in range(maxnum):
            count += 1
            print(f"Accuracy: {(correct/count):.2%}, Count: {count}", end = '\r')
            correct_answer = (i + j) % p
            lsi = [int(k) for k in str(i)]
            lsj = [int(k) for k in str(j)]
            question = lsi + [10] + lsj + [11]

            ll = len(question)
            #print("q:", question)
            pred = model.generate_greedy(question)
            answer = pred[ll: -1]
            try:
                answer = int(tokenizer.detokenize(answer))
            except:
                continue
            #print("p", pred)
            #print("a", answer)

            if answer == correct_answer:
                correct += 1
            
            
    print('\n')

In [10]:
print("Calculating accuracy for the initial model")
get_accuracy(first_model)

for i, model in enumerate(models):
    print(f"Calculating accuracy for model {intervals[i]}")
    get_accuracy(model)

print("Calculating accuracy for the final model")
get_accuracy(last_model)

Calculating accuracy for the initial model
Accuracy: 0.00%, Count: 10000

Calculating accuracy for model 200
Accuracy: 87.86%, Count: 10000

Calculating accuracy for model 400
Accuracy: 87.99%, Count: 10000

Calculating accuracy for model 600
Accuracy: 90.32%, Count: 10000

Calculating accuracy for model 800
Accuracy: 88.13%, Count: 10000

Calculating accuracy for model 1000
Accuracy: 89.24%, Count: 10000

Calculating accuracy for model 1200
Accuracy: 89.44%, Count: 10000

Calculating accuracy for model 1400
Accuracy: 90.88%, Count: 10000

Calculating accuracy for model 1600
Accuracy: 88.66%, Count: 10000

Calculating accuracy for model 1800
Accuracy: 91.55%, Count: 10000

Calculating accuracy for the final model
Accuracy: 89.75%, Count: 10000



In [ ]:
test_sentence = "1+1="
test_sentence_tokenized = tokenizer.tokenize(test_sentence)

pred = model.generate_greedy([1, 10, 2,1, 11])
print("Prediction:", pred)

Prediction: [1, 10, 2, 1, 11, 2, 2, 12]


In [ ]:
test_sentence_tokenized

[1, 10, 1, 11]